In [52]:
# init
import glob
import os
import csv
import string
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from matplotlib import pyplot as plt
%matplotlib inline

Import Data

In [33]:
header = ['class', 'document', 'feature']
pubmed = []

# import pubmed
docList = glob.glob(os.path.join(os.getcwd(), "Datasets/pubmed/", "*.txt"))

for docPath in docList:
    docName = os.path.basename(docPath).split('.')[0]
    className = docName[:3]
    with open(docPath) as doc:
        pubmed.append([className, docName, doc.read().replace('\n', ' ')])

# print(pubmed)

# export csv pubmed
with open('pubmed.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)

    # header
    writer.writerow(header)
    # row
    writer.writerows(pubmed)

Dataframe

In [34]:
# read csv
data = pd.read_csv('pubmed.csv')
data.head()

,class,document,feature
0,ALZ,ALZ1,Reduced amounts of immunoreactive somatostatin...
1,ALZ,ALZ10,Diagnostic criteria for primary neuronal degen...
2,ALZ,ALZ100,14Cacetylcholine synthesis and 14Ccarbon dioxi...
3,ALZ,ALZ1000,The pattern of reading deterioration in dement...
4,ALZ,ALZ101,Cerebral blood flow and metabolic rate of oxyg...


Preprocesing

In [53]:
features = data.loc[:, 'feature']

# punctuation removal
punctuation = []
for feature in features:
    translate = feature.translate(str.maketrans('', '', string.punctuation))
    punctuation.append(translate)

# print(punctuation)

# case folding
casefolding = []
for feature in punctuation:
    lower = feature.lower()
    casefolding.append(lower)

# print(casefolding)

# tokenization
tokenization = []
for feature in casefolding:
    token = word_tokenize(feature)
    tokenization.append(token)

# print(tokenization)

# stopwords removal
finalFeature = []
stopWords = set(stopwords.words('english'))
for token in tokenization:
    for feature in token:
        if feature not in stopWords:
            finalFeature.append(feature)

print(finalFeature)

LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/english.pickle[0m

  Searched in:
    - 'C:\\Users\\ZephZ/nltk_data'
    - 'd:\\Downloads\\Anaconda\\nltk_data'
    - 'd:\\Downloads\\Anaconda\\share\\nltk_data'
    - 'd:\\Downloads\\Anaconda\\lib\\nltk_data'
    - 'C:\\Users\\ZephZ\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - ''
**********************************************************************
